# Designing with types: Making illegal states unrepresentable
##### Encoding business logic in types

n this post, we look at a key benefit of F#, which using the type system to “make illegal states unrepresentable” (a phrase borrowed from Yaron Minsky).

Let’s look at our Contact type. Thanks to the previous refactoring, it is quite simple:

In [3]:
module EmailAddress = 

    type T = EmailAddress of string

    // create with continuation
    let createWithCont success failure (s:string) = 
        if System.Text.RegularExpressions.Regex.IsMatch(s,@"^\S+@\S+\.\S+$") 
            then success (EmailAddress s)
            else failure "Email address must contain an @ sign"

    // create directly
    let create s = 
        let success e = Some e
        let failure _  = None
        createWithCont success failure s

    // unwrap with continuation
    let apply f (EmailAddress e) = f e

    // unwrap directly
    let value e = apply id e

module ZipCode = 

    type T = ZipCode of string

    // create with continuation
    let createWithCont success failure  (s:string) = 
        if System.Text.RegularExpressions.Regex.IsMatch(s,@"^\d{5}$") 
            then success (ZipCode s) 
            else failure "Zip code must be 5 digits"
    
    // create directly
    let create s = 
        let success e = Some e
        let failure _  = None
        createWithCont success failure s

    // unwrap with continuation
    let apply f (ZipCode e) = f e

    // unwrap directly
    let value e = apply id e

module StateCode = 

    type T = StateCode of string

    // create with continuation
    let createWithCont success failure  (s:string) = 
        let s' = s.ToUpper()
        let stateCodes = ["AZ";"CA";"NY"] //etc
        if stateCodes |> List.exists ((=) s')
            then success (StateCode s') 
            else failure "State is not in list"
    
    // create directly
    let create s = 
        let success e = Some e
        let failure _  = None
        createWithCont success failure s

    // unwrap with continuation
    let apply f (StateCode e) = f e

    // unwrap directly
    let value e = apply id e

type PersonalName = 
    {
        FirstName: string;
        MiddleInitial: string option;
        LastName: string;
    }

type EmailContactInfo = 
    {
        EmailAddress: EmailAddress.T;
        IsEmailVerified: bool;
    }

type PostalAddress = 
    {
        Address1: string;
        Address2: string;
        City: string;
        State: StateCode.T;
        Zip: ZipCode.T;
    }

type PostalContactInfo = 
    {
        Address: PostalAddress;
        IsAddressValid: bool;
    }

type Contact = 
    {
        Name: PersonalName;
        EmailContactInfo: EmailContactInfo;
        PostalContactInfo: PostalContactInfo;
    }


Now let’s say that we have the following simple business rule: “A contact must have an email or a postal address”. Does our type conform to this rule?

The answer is no. The business rule implies that a contact might have an email address but no postal address, or vice versa. But as it stands, our type requires that a contact must always have both pieces of information.

The answer seems obvious – make the addresses optional, like this:

In [4]:
type Contact = 
    {
        Name: PersonalName;
        EmailContactInfo: EmailContactInfo option;
        PostalContactInfo: PostalContactInfo option;
    }

But now we have gone too far the other way. In this design, it would be possible for a contact to have neither type of address at all. But the business rule says that at least one piece of information must be present.

What’s the solution?

### Making illegal states unrepresentable

If we think about the business rule carefully, we realize that there are three possibilities:

- A contact only has an email address
- A contact only has a postal address
- A contact has both a email address and a postal address

Once it is put like this, the solution becomes obvious – use a union type with a case for each possibility.

In [6]:
type ContactInfo = 
    | EmailOnly of EmailContactInfo
    | PostOnly of PostalContactInfo
    | EmailAndPost of EmailContactInfo * PostalContactInfo

type Contact = 
    {
        Name: PersonalName;
        ContactInfo: ContactInfo;
    }

This design meets the requirements perfectly. All three cases are explicitly represented, and the fourth possible case (with no email or postal address at all) is not allowed.

Note that for the “email and post” case, I just used a tuple type for now. It’s perfectly adequate for what we need.

### Constructing a ContactInfo
Now let’s see how we might use this in practice. We’ll start by creating a new contact:

In [7]:
let contactFromEmail name emailStr = 
    let emailOpt = EmailAddress.create emailStr
    // handle cases when email is valid or invalid
    match emailOpt with
    | Some email -> 
        let emailContactInfo = 
            {EmailAddress=email; IsEmailVerified=false}
        let contactInfo = EmailOnly emailContactInfo 
        Some {Name=name; ContactInfo=contactInfo}
    | None -> None

let name = {FirstName = "A"; MiddleInitial=None; LastName="Smith"}
let contactOpt = contactFromEmail name "abc@example.com"

In this code, we have created a simple helper function contactFromEmail to create a new contact by passing in a name and email. However, the email might not be valid, so the function has to handle both cases, which it does by returning a `Contact option`, not a `Contact`

### Updating a ContactInfo

Now if we need to add a postal address to an existing ContactInfo, we have no choice but to handle all three possible cases:

- If a contact previously only had an email address, it now has both an email address and a postal address, so return a contact using the EmailAndPost case.
- If a contact previously only had a postal address, return a contact using the PostOnly case, replacing the existing address.
- If a contact previously had both an email address and a postal address, return a contact with using the EmailAndPost case, replacing the existing address.

So here’s a helper method that updates the postal address. You can see how it explicitly handles each case.

In [8]:
let updatePostalAddress contact newPostalAddress = 
    let {Name=name; ContactInfo=contactInfo} = contact
    let newContactInfo =
        match contactInfo with
        | EmailOnly email ->
            EmailAndPost (email,newPostalAddress) 
        | PostOnly _ -> // ignore existing address
            PostOnly newPostalAddress 
        | EmailAndPost (email,_) -> // ignore existing address
            EmailAndPost (email,newPostalAddress) 
    // make a new contact
    {Name=name; ContactInfo=newContactInfo}

And here is the code in use:

In [9]:
let contact = contactOpt.Value   // see warning about option.Value below
let newPostalAddress = 
    let state = StateCode.create "CA"
    let zip = ZipCode.create "97210"
    {   
        Address = 
            {
            Address1= "123 Main";
            Address2="";
            City="Beverly Hills";
            State=state.Value; // see warning about option.Value below
            Zip=zip.Value;     // see warning about option.Value below
            }; 
        IsAddressValid=false
    }
let newContact = updatePostalAddress contact newPostalAddress


> WARNING: I am using option.Value to extract the contents of an option in this code. This is ok when playing around interactively but is extremely bad practice in production code! You should always use matching to handle both cases of an option.


### Why bother to make these complicated types? 
At this point, you might be saying that we have made things unnecessarily complicated. I would answer with these points:

First, the business logic is complicated. There is no easy way to avoid it. If your code is not this complicated, you are not handling all the cases properly.

Second, if the logic is represented by types, it is automatically self documenting. You can look at the union cases below and immediate see what the business rule is. You do not have to spend any time trying to analyze any other code.


In [10]:
type ContactInfo = 
    | EmailOnly of EmailContactInfo
    | PostOnly of PostalContactInfo
    | EmailAndPost of EmailContactInfo * PostalContactInfo

Finally, if the logic is represented by a type, any changes to the business rules will immediately create breaking changes, which is a generally a good thing.

In the next post, we’ll dig deeper into the last point. As you try to represent business logic using types, you may suddenly find that can gain a whole new insight into the domain.
